In [1]:
import torch
assert torch.cuda.is_available()
print(torch.version, torch.version.cuda)

<module 'torch.version' from '/home/usr/miniforge3/lib/python3.12/site-packages/torch/version.py'> 12.4


In [2]:
import torch
from utils.config import ModelConfig
from models.unet import UNetWithCrossAttention
from models.diffusion import Diffusion
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from utils.SoundDataset import SoundDataset
from torch.utils.data import DataLoader

RuntimeError: Detected that PyTorch and TorchAudio were compiled with different CUDA versions. PyTorch has CUDA version 12.4 whereas TorchAudio has CUDA version 12.1. Please install the TorchAudio version that matches your PyTorch version.

In [ ]:
def load_snapshot(snapshot_path, raw_model):
    snapshot = torch.load(snapshot_path)
    raw_model.load_state_dict(snapshot["MODEL_STATE"])
    train_losses = snapshot["TRAIN_LOSSES"]
    val_losses = snapshot["VAL_LOSSES"]

    return raw_model, train_losses, val_losses

In [ ]:
image_path = "data/images"
sound_path = "data/sounds"

data = SoundDataset(image_path, sound_path)

val_loader = DataLoader(data, 
                        batch_size=8,
                        pin_memory=True,
                        shuffle=False, 
                        drop_last=True)

In [ ]:
# from archisound import ArchiSound

device = "cuda" if torch.cuda.is_available() else "cpu"

# autoencoder = ArchiSound.from_pretrained("dmae1d-ATC64-v2").to(device)

from models.AudioEncoder import AudioEncoder

encode = AudioEncoder("facebook/wav2vec2-base-960h",
                                    device,
                                    48000)

In [ ]:
config = ModelConfig({"image_size": 128, "audio_ctx_dim": 768})

# Инициализация
diffusion = Diffusion(timesteps=1000, image_size=128, device=device)
model = UNetWithCrossAttention(config)

In [ ]:
model, train_losses, val_losses = load_snapshot("result.pt", model)

In [ ]:
plt.plot(train_losses[:], label='train')
plt.plot(val_losses[:], label='val')
plt.grid()
plt.legend()

In [ ]:
val_losses[20:]

In [ ]:
train_losses[20:]

In [ ]:
audio,image = next(iter(val_loader))
audio_embeds = encode(audio.to(device))

model.to(device)
model.eval()
generated_image = diffusion.reverse_process(
    model,
    audio_embeds,
    guidance_scale=7.5,
    batch_size=8,
    use_ddim=True,
    timesteps=100  #число шагов
)

In [ ]:
plt.imshow(torch.permute(generated_image[0].cpu(), (1,2,0)).numpy())

In [ ]:
plt.imshow(torch.permute(image[0], (1,2,0)))

In [ ]:
# модель сошлась на lr=0.0005 на 30 эпохах. Потери составили ~0.68
# размерность UNet:
# DownBlock(3, 32, 0.3),
# DownBlock(32, 64, 0.3),
# DownBlock(64, 128, 0.3),
# DownBlock(128, 256, 0.3)

# попробуем увеличить размер UNet